# Multimodal classification

Multimodal classification by a dual branch modified VGG network
Created on 30/08/2023

In [1]:
from sklearn.metrics import roc_curve
import numpy as np
import os
from keras.models import Sequential, Model, load_model
import pandas as pd
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input, concatenate
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix as cm, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import itertools
from sklearn.model_selection import GroupShuffleSplit
import tensorflow as tf
#import tensorflow_addons as tfa
from sklearn.utils import class_weight
from keras.optimizers import Adam, SGD
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from itertools import product
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
import pickle
from sklearn.utils import compute_class_weight
import math

In [2]:
# Multimodal Folders
ecg_folder = 'ecg'
pcg_folder = 'pcg'
batch_size = 32


In [3]:
# Loading pcg dataset info

train_df_ecg = pd.read_csv(f"./{ecg_folder}/train/dataset.csv",usecols=range(1,3))
test_df_ecg = pd.read_csv(f"./{ecg_folder}/test/dataset.csv",usecols=range(1,3))


train_df_ecg['label']=train_df_ecg['label'].astype(str)
test_df_ecg['label']=test_df_ecg['label'].astype(str)

train_df_ecg['group'] = train_df_ecg['filename'].apply(lambda x: x.split('_')[0])
test_df_ecg['group'] = test_df_ecg['filename'].apply(lambda x: x.split('_')[0])

In [4]:
# Loading ecg dataset info
train_df_pcg = pd.read_csv(f"./{pcg_folder}/train/dataset.csv",usecols=range(1,3))
test_df_pcg = pd.read_csv(f"./{pcg_folder}/test/dataset.csv",usecols=range(1,3))


train_df_pcg['label']=train_df_pcg['label'].astype(str)
test_df_pcg['label']=test_df_pcg['label'].astype(str)

train_df_pcg['group'] = train_df_pcg['filename'].apply(lambda x: x.split('_')[0])
test_df_pcg['group'] = test_df_pcg['filename'].apply(lambda x: x.split('_')[0])

In [5]:
# Sorting by filename so the datasets are guaranteed to be aligned
train_df_pcg.sort_values('filename', inplace = True)
test_df_pcg.sort_values('filename', inplace = True)
train_df_ecg.sort_values('filename', inplace = True)
test_df_ecg.sort_values('filename', inplace = True)

In [6]:
# Making sure the dataframe have exactly the same filenames and are ligned up properly

print (train_df_pcg['filename'].tolist() ==train_df_ecg['filename'].tolist())

print (test_df_pcg['filename'].tolist() ==test_df_ecg['filename'].tolist())


True
True


In [7]:
# Creating ImageDataGenerators (no image augmentation is used),

train_datagen_ecg = ImageDataGenerator(preprocessing_function=preprocess_input)
train_datagen_pcg = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen_pcg = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen_ecg = ImageDataGenerator(preprocessing_function=preprocess_input)


In [8]:
# Creating the generator flows from the dataframe info (mapping the dfs to the respective folder)
ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
    dataframe=train_df_ecg,
    directory=f'{ecg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')


ecg_test_gen = test_datagen_ecg.flow_from_dataframe(
    dataframe=test_df_ecg,
    directory=f'{ecg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

pcg_test_gen = test_datagen_pcg.flow_from_dataframe(
    dataframe=test_df_pcg,
    directory=f'{pcg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
    dataframe=train_df_pcg,
    directory=f'{pcg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')


Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 1793 validated image filenames belonging to 2 classes.


# Experiments

In each experiment GridSearchCV can/will be applied to discover the best learning parameters. Different model variations will be compared on sample-wise classification. Average recordwise results are obtained. The model with best 

### 1. Individual model from scratch (no weight initialization) 

### 2. Multimodal Learning from scratch (no weight initialization)

### 3. Multimodal Learning with pre-trained VGG (on Imagenet Only)

### 4. Multimodal Learning with pre-trained VGG (on Imagenet + bigger individual datasets)

### 5. Multimodal Learning with pre-trained VGG (on Imagenet + finetuned on individual datasets)

In [9]:
# Define the parameter grid
param_grid = {
    'batch_size': [ 32],
    'epochs': [100],
    'optimizer': ['adam'],
    'learning_rate': [ 0.0005],
    'dropout':[ 0.5],
    'neurons':[128],
    'class_weights':[True]
}


# Store results
results = []



sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )


In [10]:
def calculate_scores(classes, generator, model, steps):
    y_true = []
    
    for i in range(steps):
    # Get the next batch
        batch = next(generator)

        # The sample classes (labels) are in the second element of the batch
        sample_classes = batch[1]
        
        # Append the classes to the list
        y_true.extend(sample_classes)
    #print (y_true)
    predictions = model.predict(generator, steps = steps )
    threshold = 0.5
    predicted_classes = (predictions > threshold).astype(int)
    
    f1_value = f1_score(y_true,predicted_classes, average='binary' )
    acc = accuracy_score(y_true, predicted_classes)
    auc = roc_auc_score(y_true, predictions) 
    precision = precision_score(y_true, predicted_classes)
    recall = recall_score(y_true, predicted_classes)
    #print ("model.evaluate :", model.evaluate(generator, steps = steps))
    return f1_value, acc, auc, precision, recall, predictions, y_true
    

In [24]:
def multimodal_imagenet(learning_rate = 0.0001, dropout = 0.5, neurons = 128, optimizer='adam', weights='imagenet'):
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
        
    base_model = VGG16(weights=weights, include_top=False)
    
    for layer in base_model.layers:
        layer.trainable = False



    base_model_2 = VGG16(weights=weights, include_top=False)
    base_model_2._name = 'vgg16_2'
    # Freeze the convolutional layers of the base model
    for layer in base_model_2.layers:
        layer.trainable = False



    # Create two separate input layers
    inputA = Input(shape=(224, 224, 3))
    inputB = Input(shape=(224, 224, 3))

    # Create the first branch of the CNN
    branch1 = base_model(inputA)
    branch1 = Flatten()(branch1)
    branch1 = Dense(neurons, activation='relu')(branch1)
    #branch1 = Dropout(dropout)(branch1)

    # Create the second branch of the CNN
    branch2 = base_model_2(inputB)
    branch2 = Flatten()(branch2)
    branch2 = Dense(neurons, activation='relu')(branch2)
    #branch2 = Dropout(dropout)(branch2)

    # Merge the outputs of the two branches
    merged = concatenate([branch1, branch2])

    # Add a dense layer and a sigmoid output layer
    output = Flatten()(merged)
    output = Dense(neurons, activation='relu')(output)
    output = Dropout(dropout)(output)
    output = Dense(1, activation='sigmoid')(output)
    
    # Create the model
    modelVGG = Model(inputs=[inputA, inputB], outputs=output)
    modelVGG.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return modelVGG


In [12]:
def multimodal_3(learning_rate = 0.0001, dropout = 0.5, neurons = 128, optimizer='adam', weights=None):
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
        
    base_model = VGG16(weights=weights, include_top=False)



    base_model_2 = VGG16(weights=weights, include_top=False)
    base_model_2._name = 'vgg16_2'
    # Freeze the convolutional layers of the base model
    for layer in base_model_2.layers:
        #layer.trainable = False
        layer._name = layer.name + '_2'
        layer._name =  layer._name + '_2'


    # Create two separate input layers
    inputA = Input(shape=(224, 224, 3))
    inputB = Input(shape=(224, 224, 3))

    # Create the first branch of the CNN
    branch1 = base_model(inputA)
    branch1 = Flatten()(branch1)
    branch1 = Dense(neurons, activation='relu')(branch1)
    #branch1 = Dropout(dropout)(branch1)

    # Create the second branch of the CNN
    branch2 = base_model_2(inputB)
    branch2 = Flatten()(branch2)
    branch2 = Dense(neurons, activation='relu')(branch2)
    #branch2 = Dropout(dropout)(branch2)

    # Merge the outputs of the two branches
    merged = concatenate([branch1, branch2])

    # Add a dense layer and a sigmoid output layer
    output = Flatten()(merged)
    output = Dense(neurons, activation='relu')(output)
    output = Dropout(dropout)(output)
    output = Dense(1, activation='sigmoid')(output)
    
    # Create the model
    modelVGG = Model(inputs=[inputA, inputB], outputs=output)
    modelVGG.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return modelVGG


In [13]:
ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
    dataframe=train_df_ecg,
    directory=f'{ecg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
    dataframe=train_df_pcg,
    directory=f'{pcg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

ecg_test_gen = test_datagen_ecg.flow_from_dataframe(
    dataframe=test_df_ecg,
    directory=f'{ecg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_test_gen = test_datagen_pcg.flow_from_dataframe(
    dataframe=test_df_pcg,
    directory=f'{pcg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

def pair_generator(gen1, gen2):
    dataset_size = gen1.n
    steps_per_epoch = gen1.batch_size
    steps_per_epoch = dataset_size // batch_size
    while True:
        # Get the next batch from each generator
        x1, y1 = next(gen1)
        x2, y2 = next(gen2)
        # Yield a tuple containing the inputs and labels for each branch
        yield [[x1, x2], y1]


# Create a paired generator using the two separate generators
paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


    

Found 1793 validated image filenames belonging to 2 classes.
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.


In [14]:
# Load the pre-trained model
ecg_pre_trained = load_model('ecg_only_.h5')

# Create a new input layer with the same shape as the original input
input_tensor = Input(shape=ecg_pre_trained.input_shape[1:])

# Define the submodel with all layers except the last one
ecg_model_layers = ecg_pre_trained.layers[:-1]
x = input_tensor
for layer in ecg_model_layers:
    x = layer(x)

# Create the new model
ecg_model = Model(inputs=input_tensor, outputs=x)

# Freeze the layers in the new model
for layer in ecg_model.layers:
    layer.trainable = False

In [15]:
# Load the pre-trained model
pcg_pre_trained = load_model('pcg_only_.h5')

# Create a new input layer with the same shape as the original input
input_tensor = Input(shape=pcg_pre_trained.input_shape[1:])

# Define the submodel with all layers except the last one
pcg_model_layers = pcg_pre_trained.layers[:-1]
x = input_tensor
for layer in pcg_model_layers:
    x = layer(x)

# Create the new model
pcg_model = Model(inputs=input_tensor, outputs=x)

# Freeze the layers in the new model
for layer in pcg_model.layers:
    layer.trainable = False
    

In [16]:
def multimodal_4(learning_rate = 0.0001, dropout = 0.5, neurons = 128, optimizer='adam', weights=None):
    # CONV LAYERS set to imagenet weights
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
        
    base_model = VGG16(weights=weights, include_top=False)



    base_model_2 = VGG16(weights=weights, include_top=False)

    # Freeze the convolutional layers of the base model
    for layer in base_model.layers:
        layer.trainable = False


    #base_model_2 = pcg_pre_trained
    base_model_2._name = 'vgg16_2'
    # Freeze the convolutional layers of the base model
    for layer in base_model_2.layers:
        layer.trainable = False
        layer._name = layer.name + '_2'
        layer._name =  layer._name + '_2'


    # Create two separate input layers
    inputA = Input(shape=(224, 224, 3))
    inputB = Input(shape=(224, 224, 3))

    branch1 = base_model(inputA)
    branch1 = Flatten()(branch1)
    branch1 = Dense(neurons, activation='relu')(branch1)
    #branch1 = Dropout(dropout)(branch1)

    # Create the second branch of the CNN
    branch2 = base_model_2(inputB)
    branch2 = Flatten()(branch2)
    branch2 = Dense(neurons, activation='relu')(branch2)

    # Merge the outputs of the two branches
    merged = concatenate([branch1, branch2])

    # Add a dense layer and a sigmoid output layer
    output = Flatten()(merged)
    output = Dense(neurons, activation='relu')(output)
    output = Dropout(dropout)(output)
    output = Dense(1, activation='sigmoid')(output)
    
    # Create the model
    modelVGG = Model(inputs=[inputA, inputB], outputs=output)
    modelVGG.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return modelVGG


In [17]:
param_grid = {
    'batch_size': [ 8],
    'epochs': [30],
    'optimizer': ['adam'],
    'learning_rate': [ 0.000001, 0.00001, 0.0005],
    'dropout':[  0.5],
    'neurons':[128],
    'class_weights':[True]
}

In [38]:
##### # For each combination

sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )

for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    use_class_weights = param_dict['class_weights']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_ecg['filename'], train_df_ecg['label'], groups=train_df_ecg['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = multimodal_3(**param_dict_filtered)


        ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[train_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[train_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        ecg_test_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[test_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_test_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[test_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        
        paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        train_group = train_df_ecg.iloc[train_indices]['group']
        val_group = train_df_ecg.iloc[test_indices]['group']
        
        if use_class_weights:
            y_train = ecg_train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)
            
            
            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        else:
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        
        #paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        #paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))
        
        #test_predictions = model.predict(test_gen)
        
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['accuracy']
        epoch_val_accuracy = history.history['val_accuracy']
        epoch_val_losses = history.history['val_loss']

        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    detailed_results_df.to_csv('multimodal_vgg.csv')
    with open ('ecg_pcg'+dt_string+'multimodal_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 30, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 128, 'class_weights': True}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/30
180/180 [==============================] - 69s 354ms/step - loss: 0.7294 - accuracy: 0.5541 - auc: 0.5041 - val_loss: 0.7244 - val_accuracy: 0.2750 - val_auc: 0.5626
Epoch 2/30
180/180 [==============================] - 65s 360ms/step - loss: 0.7022 - accuracy: 0.4836 - auc: 0.4996 - val_loss: 0.7126 - val_accuracy: 0.2861 - val_auc: 0.6826
Epoch 3/30
180/180 [==============================] - 64s 353ms/step - loss: 0.6934 - accuracy: 0.5345 - auc: 0.5349 - val_loss: 0.7083 - val_accuracy: 0.3417 - val_auc: 0.7127
Epoch 4/30
180/180 [==============================] - 

Epoch 11/30
178/178 [==============================] - 59s 331ms/step - loss: 0.5538 - accuracy: 0.7331 - auc: 0.8066 - val_loss: 0.6470 - val_accuracy: 0.6836 - val_auc: 0.7367
Epoch 12/30
178/178 [==============================] - 59s 331ms/step - loss: 0.5446 - accuracy: 0.7465 - auc: 0.8160 - val_loss: 0.6437 - val_accuracy: 0.6863 - val_auc: 0.7386
Epoch 13/30
178/178 [==============================] - 59s 331ms/step - loss: 0.5276 - accuracy: 0.7570 - auc: 0.8314 - val_loss: 0.6549 - val_accuracy: 0.6863 - val_auc: 0.7442
Epoch 14/30
178/178 [==============================] - 59s 331ms/step - loss: 0.5208 - accuracy: 0.7627 - auc: 0.8364 - val_loss: 0.6456 - val_accuracy: 0.6863 - val_auc: 0.7481
Epoch 15/30
178/178 [==============================] - 59s 331ms/step - loss: 0.5011 - accuracy: 0.7711 - auc: 0.8509 - val_loss: 0.6252 - val_accuracy: 0.6944 - val_auc: 0.7500
Epoch 16/30
178/178 [==============================] - 59s 331ms/step - loss: 0.4940 - accuracy: 0.7880 - auc:

181/181 [==============================] - 60s 329ms/step - loss: 0.4315 - accuracy: 0.8182 - auc: 0.9044 - val_loss: 0.5522 - val_accuracy: 0.7528 - val_auc: 0.7473
Epoch 24/30
181/181 [==============================] - 60s 329ms/step - loss: 0.4126 - accuracy: 0.8341 - auc: 0.9172 - val_loss: 0.5624 - val_accuracy: 0.7415 - val_auc: 0.7515
Epoch 25/30
181/181 [==============================] - 60s 329ms/step - loss: 0.4048 - accuracy: 0.8341 - auc: 0.9180 - val_loss: 0.5605 - val_accuracy: 0.7443 - val_auc: 0.7546
Epoch 26/30
181/181 [==============================] - 60s 329ms/step - loss: 0.3983 - accuracy: 0.8508 - auc: 0.9215 - val_loss: 0.5574 - val_accuracy: 0.7301 - val_auc: 0.7635
Epoch 27/30
181/181 [==============================] - 60s 329ms/step - loss: 0.3795 - accuracy: 0.8626 - auc: 0.9340 - val_loss: 0.5387 - val_accuracy: 0.7472 - val_auc: 0.7655
Epoch 28/30
181/181 [==============================] - 60s 329ms/step - loss: 0.3603 - accuracy: 0.8695 - auc: 0.9402 - va

181/181 [==============================] - 63s 342ms/step - loss: 0.7388 - accuracy: 0.5444 - auc: 0.4926 - val_loss: 0.6942 - val_accuracy: 0.4615 - val_auc: 0.6152
Epoch 2/30
181/181 [==============================] - 59s 329ms/step - loss: 0.7001 - accuracy: 0.5361 - auc: 0.5078 - val_loss: 0.6981 - val_accuracy: 0.4758 - val_auc: 0.5946
Epoch 3/30
181/181 [==============================] - 59s 329ms/step - loss: 0.6896 - accuracy: 0.5451 - auc: 0.5577 - val_loss: 0.6932 - val_accuracy: 0.4986 - val_auc: 0.6164
Epoch 4/30
181/181 [==============================] - 60s 329ms/step - loss: 0.6748 - accuracy: 0.5700 - auc: 0.6128 - val_loss: 0.7094 - val_accuracy: 0.4644 - val_auc: 0.6134
Epoch 5/30
181/181 [==============================] - 59s 329ms/step - loss: 0.6569 - accuracy: 0.6110 - auc: 0.6577 - val_loss: 0.6968 - val_accuracy: 0.5128 - val_auc: 0.6205
Epoch 6/30
181/181 [==============================] - 59s 329ms/step - loss: 0.6360 - accuracy: 0.6546 - auc: 0.7031 - val_los

{'batch_size': 8, 'epochs': 30, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 128, 'class_weights': True}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/30
180/180 [==============================] - 61s 332ms/step - loss: 0.7678 - accuracy: 0.5429 - auc: 0.5031 - val_loss: 0.7032 - val_accuracy: 0.2750 - val_auc: 0.5061
Epoch 2/30
180/180 [==============================] - 59s 329ms/step - loss: 0.7016 - accuracy: 0.4920 - auc: 0.4769 - val_loss: 0.7106 - val_accuracy: 0.2750 - val_auc: 0.6546
Epoch 3/30
180/180 [==============================] - 59s 329ms/step - loss: 0.7000 - accuracy: 0.5234 - auc: 0.4757 - val_loss: 0.6920 - val_accuracy: 0.5639 - val_auc: 0.7706
Epoch 4/30
180/180 [==============================] - 

...

c:\users\helde\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 1442 validated image filenames belonging to 2 classes.
Found 1442 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Epoch 1/30
181/181 [==============================] - 60s 326ms/step - loss: 2536.8335 - accuracy: 0.5083 - auc: 0.4663 - val_loss: 0.6942 - val_accuracy: 0.3048 - val_auc: 0.5000
Epoch 2/30
181/181 [==============================] - 59s 325ms/step - loss: 0.6941 - accuracy: 0.5270 - auc: 0.4572 - val_loss: 0.6932 - val_accuracy: 0.3048 - val_auc: 0.5000
Epoch 3/30
181/181 [==============================] - 59s 325ms/step - loss: 0.6935 - accuracy: 0.4875 - auc: 0.4678 - val_loss: 0.6937 - val_accuracy: 0.3048 - val_auc: 0.5000
Epoch 4/30
181/181 [==============================] - 59s 325ms/step - loss: 0.6935 - accuracy: 0.3807 - auc: 0.4751 - val_loss: 0.6947 - val_accuracy: 0.3048 - val_auc: 0.5000
Epoch 5/30
181/181 [===========================

c:\users\helde\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


44/44 [==============================] - 5s 109ms/step
0.0 0.2871012482662968 0.5 0.0 0.0
0.0 0.30484330484330485 0.5 0.0 0.0
                                              params  fold_number  \
0  {'batch_size': 8, 'epochs': 30, 'optimizer': '...            1   
1  {'batch_size': 8, 'epochs': 30, 'optimizer': '...            2   
2  {'batch_size': 8, 'epochs': 30, 'optimizer': '...            3   
3  {'batch_size': 8, 'epochs': 30, 'optimizer': '...            4   
4  {'batch_size': 8, 'epochs': 30, 'optimizer': '...            5   

                                                loss  \
0  [636.9083862304688, 0.693562388420105, 0.69332...   
1  [6926.68798828125, 0.6941448450088501, 0.69411...   
2  [110.57380676269531, 0.6938509941101074, 0.693...   
3  [135.67568969726562, 1.0141606330871582, 0.693...   
4  [2536.83349609375, 0.6940905451774597, 0.69353...   

                                           acc_train  \
0  [0.47313329577445984, 0.4773203134536743, 0.38...   
1  [0.4957

c:\users\helde\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Load the pre-trained model
ecg_pre_trained = load_model('ecg_only_.h5')


In [15]:
# Load the pre-trained model
pcg_pre_trained = load_model('pcg_only_.h5')

In [18]:
param_grid = {
    'batch_size': [ 8],
    'epochs': [20],
    'optimizer': ['adam'],
    'learning_rate': [ 0.000001, 0.00001, 0.0005],
    'dropout':[  0.5],
    'neurons':[128],
    'class_weights':[False]
}

In [19]:
def multimodal_5(learning_rate = 0.0001, dropout = 0.5, neurons = 128, optimizer='adam', weights=None):
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
    
    model = Sequential()
    model2 = Sequential()
    base_model = load_model('ecg_only_finetuned.h5')



    base_model_2 = load_model('pcg_only_finetune_2.h5')
    

    
    
    
    
    base_model_2._name = 'vgg16_2'
    
    
    for layer in base_model.layers[:-3]: 
        layer.trainable = False
        model.add(layer)
        
    # Freeze the convolutional layers of the base model
    for layer in base_model_2.layers:
        layer.trainable = False
        layer._name = layer.name + '_2'
        layer._name =  layer._name + '_2'
        model2.add(layer)

    # Create two separate input layers
    inputA = Input(shape=(224, 224, 3))
    inputB = Input(shape=(224, 224, 3))

    # Create the first branch of the CNN
    branch1 = model(inputA)
    branch1 = Flatten()(branch1)
    branch1 = Dense(neurons, activation='relu')(branch1)
    #branch1 = Dropout(dropout)(branch1)
    
    #print (model2.summary())
        
    
    # Create the second branch of the CNN
    branch2 = model2(inputB)
    branch2 = Flatten()(branch2)
    branch2 = Dense(neurons, activation='relu')(branch2)
    #branch2 = Dropout(dropout)(branch2)

    # Merge the outputs of the two branches
    merged = concatenate([branch1, branch2])

    # Add a dense layer and a sigmoid output layer
    output = Flatten()(merged)
    output = Dense(neurons, activation='relu')(output)
    output = Dropout(dropout)(output)
    output = Dense(1, activation='sigmoid')(output)
    
    # Create the model
    modelVGG = Model(inputs=[inputA, inputB], outputs=output)
    modelVGG.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    #print (modelVGG.summary())
    return modelVGG


In [22]:
param_grid = {
    'batch_size': [ 8],
    'epochs': [20],
    'optimizer': ['adam'],
    'learning_rate': [ 0.000001, 0.00001, 0.00005],
    'dropout':[  0.5],
    'neurons':[32],
    'class_weights':[False, True]
}

In [23]:
##### # For each combination

sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )

for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    use_class_weights = param_dict['class_weights']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_ecg['filename'], train_df_ecg['label'], groups=train_df_ecg['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = multimodal_5(**param_dict_filtered)


        ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[train_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[train_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        ecg_test_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[test_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_test_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[test_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        
        paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        train_group = train_df_ecg.iloc[train_indices]['group']
        val_group = train_df_ecg.iloc[test_indices]['group']
        
        if use_class_weights:
            y_train = ecg_train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)
            
            
            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        else:
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        
        #paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        #paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))
        
        #test_predictions = model.predict(test_gen)
        
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['accuracy']
        epoch_val_accuracy = history.history['val_accuracy']
        epoch_val_losses = history.history['val_loss']

        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    detailed_results_df.to_csv('multimodal_vgg.csv')
    with open ('ecg_pcg'+dt_string+'finetuned_multimodal_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/20
180/180 [==============================] - 27s 143ms/step - loss: 0.8461 - accuracy: 0.5010 - auc: 0.4880 - val_loss: 0.6580 - val_accuracy: 0.6222 - val_auc: 0.5996
Epoch 2/20
180/180 [==============================] - 25s 140ms/step - loss: 0.8151 - accuracy: 0.5290 - auc: 0.5037 - val_loss: 0.6391 - val_accuracy: 0.6500 - val_auc: 0.5936
Epoch 3/20
180/180 [==============================] - 25s 140ms/step - loss: 0.7688 - accuracy: 0.5478 - auc: 0.5168 - val_loss: 0.6259 - val_accuracy: 0.6722 - val_auc: 0.5914
Epoch 4/20
180/180 [==============================] - 

47/47 [==============================] - 5s 107ms/step
0.8284376330353341 0.7161971830985916 0.6385131890941592 0.7337858220211161 0.9511241446725318
0.7823129251700682 0.6568364611260054 0.6154294597745822 0.6784660766961652 0.9236947791164659
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
-----------------FOLD 2-----------------
Found 1441 validated image filenames belonging to 2 classes.
Found 1441 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 27s 142ms/step - loss: 0.9842 - accuracy: 0.4358 - auc: 0.5070 - val_loss: 0.9286 - val_accuracy: 0.3665 - val_auc: 0.4344
Epoch 2/20
181/181 [==============================] - 25s 141ms/step - loss: 0.9001 - accuracy: 0.4608 - auc: 0.5237 - val_loss: 0.8670 - val_accuracy: 0.4176 - va

180/180 [==============================] - 26s 142ms/step - loss: 0.6155 - accuracy: 0.6741 - auc: 0.6278 - val_loss: 0.6544 - val_accuracy: 0.6919 - val_auc: 0.5026
Epoch 20/20
45/45 [==============================] - 5s 107ms/step
0.8343245539507221 0.7284122562674095 0.7233759842519685 0.7339312406576981 0.9665354330708661
0.8161888701517707 0.6946778711484594 0.5047184405940595 0.7181008902077152 0.9453125
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
-----------------FOLD 4-----------------
Found 1442 validated image filenames belonging to 2 classes.
Found 1442 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 29s 152ms/step - loss: 0.8428 - accuracy: 0.5284 - auc: 0.5024 - val_loss: 0.7026 - val_accuracy: 0.5385 - val_auc: 0

{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/20
180/180 [==============================] - 27s 143ms/step - loss: 0.8290 - accuracy: 0.5394 - auc: 0.5482 - val_loss: 0.7452 - val_accuracy: 0.5028 - val_auc: 0.4522
Epoch 2/20
180/180 [==============================] - 25s 141ms/step - loss: 0.8222 - accuracy: 0.5429 - auc: 0.5463 - val_loss: 0.7435 - val_accuracy: 0.4944 - val_auc: 0.4587
Epoch 3/20
180/180 [==============================] - 25s 141ms/step - loss: 0.8045 - accuracy: 0.5311 - auc: 0.5410 - val_loss: 0.7392 - val_accuracy: 0.4972 - val_auc: 0.4659
Epoch 4/20
180/180 [==============================] - 2

47/47 [==============================] - 5s 107ms/step
0.6253908692933083 0.578169014084507 0.7368435307819399 0.8680555555555556 0.4887585532746823
0.57356608478803 0.5415549597855228 0.6265060240963856 0.756578947368421 0.46184738955823296
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
-----------------FOLD 2-----------------
Found 1441 validated image filenames belonging to 2 classes.
Found 1441 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 27s 143ms/step - loss: 0.9476 - accuracy: 0.5586 - auc: 0.5044 - val_loss: 0.7509 - val_accuracy: 0.5682 - val_auc: 0.3947
Epoch 2/20
181/181 [==============================] - 25s 141ms/step - loss: 0.8971 - accuracy: 0.5573 - auc: 0.5208 - val_loss: 0.7459 - val_accuracy: 0.5739 - val_au

...

180/180 [==============================] - 25s 142ms/step - loss: 0.0933 - accuracy: 0.9784 - auc: 0.9973 - val_loss: 0.5116 - val_accuracy: 0.8571 - val_auc: 0.8846
Epoch 20/20
45/45 [==============================] - 5s 116ms/step
0.9965601965601966 0.995125348189415 0.9998453318335208 0.9950932286555446 0.9980314960629921
0.9021956087824351 0.8627450980392157 0.8858292079207921 0.9224489795918367 0.8828125
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
-----------------FOLD 4-----------------
Found 1442 validated image filenames belonging to 2 classes.
Found 1442 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 29s 155ms/step - loss: 0.7148 - accuracy: 0.6630 - auc: 0.5272 - val_loss: 0.5814 - val_accuracy: 0.6923 - val_auc: 0.

{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/20
180/180 [==============================] - 28s 151ms/step - loss: 0.8650 - accuracy: 0.6246 - auc: 0.5821 - val_loss: 0.6433 - val_accuracy: 0.6028 - val_auc: 0.6315
Epoch 2/20
180/180 [==============================] - 28s 155ms/step - loss: 0.6062 - accuracy: 0.6748 - auc: 0.7310 - val_loss: 0.5953 - val_accuracy: 0.6667 - val_auc: 0.7007
Epoch 3/20
180/180 [==============================] - 29s 161ms/step - loss: 0.5487 - accuracy: 0.7369 - auc: 0.7948 - val_loss: 0.5527 - val_accuracy: 0.7028 - val_auc: 0.7517
Epoch 4/20
180/180 [==============================] - 2

47/47 [==============================] - 5s 118ms/step
0.9916297390448056 0.9880281690140845 0.999817792781147 0.9990079365079365 0.9843597262952102
0.8737864077669903 0.8257372654155496 0.8385801269594506 0.8458646616541353 0.9036144578313253
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
-----------------FOLD 2-----------------
Found 1441 validated image filenames belonging to 2 classes.
Found 1441 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Found 352 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 30s 156ms/step - loss: 0.7320 - accuracy: 0.5961 - auc: 0.5976 - val_loss: 0.5826 - val_accuracy: 0.7159 - val_auc: 0.8102
Epoch 2/20
181/181 [==============================] - 27s 151ms/step - loss: 0.5878 - accuracy: 0.6613 - auc: 0.7443 - val_loss: 0.5446 - val_accuracy: 0.7216 - val_

180/180 [==============================] - 27s 147ms/step - loss: 0.1369 - accuracy: 0.9582 - auc: 0.9937 - val_loss: 0.5374 - val_accuracy: 0.8739 - val_auc: 0.8949
Epoch 20/20
45/45 [==============================] - 5s 112ms/step
0.9895678092399404 0.9853760445682451 0.9995125609298837 0.9989969909729187 0.9803149606299213
0.9054325955734406 0.8683473389355743 0.8978186881188119 0.9336099585062241 0.87890625
{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
-----------------FOLD 4-----------------
Found 1442 validated image filenames belonging to 2 classes.
Found 1442 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Found 351 validated image filenames belonging to 2 classes.
Epoch 1/20
181/181 [==============================] - 28s 148ms/step - loss: 0.7607 - accuracy: 0.5992 - auc: 0.5790 - val_loss: 0.6445 - val_accuracy: 0.5926 - val_auc: 0

[5 rows x 22 columns]
date and time = 27_09_2023__01_39_47


In [31]:
##### # For each combination
param_dict = {'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )
detailed_results = []
epoch_info =[]
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
use_class_weights = param_dict['class_weights']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}

print(param_dict)
#print (f'-----------------FOLD {fold_idx}-----------------')

model = multimodal_5(**param_dict_filtered)


ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
    dataframe=train_df_ecg,
    directory=f'{ecg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
    dataframe=train_df_pcg,
    directory=f'{pcg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

ecg_test_gen = test_datagen_ecg.flow_from_dataframe(
    dataframe=test_df_ecg,
    directory=f'{ecg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_test_gen = test_datagen_pcg.flow_from_dataframe(
    dataframe=test_df_pcg,
    directory=f'{pcg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')


paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)

train_group = train_df_ecg['group']
val_group = test_df_ecg['group']

if use_class_weights:
    y_train = ecg_train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
else:
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))


#paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
#paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))


ecg_train_gen.reset()

pcg_train_gen.reset()

ecg_test_gen.reset()

pcg_test_gen.reset()

paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))

#test_predictions = model.predict(test_gen)



print ( train_f1, train_acc, train_auc, train_precision,train_recall)

print (test_f1, test_acc, test_auc, test_precision, test_recall)



epoch_losses = history.history['loss']

epoch_accuracy = history.history['accuracy']
epoch_val_accuracy = history.history['val_accuracy']
epoch_val_losses = history.history['val_loss']

epoch_info.append({
    'params': param_dict,
    'fold_number': -1,
    'loss': epoch_losses,
    #'val_loss':epoch_val_losses,
    'acc_train': epoch_accuracy,
    #'f1_train': epoch_f1_macro,
    'acc_val': epoch_val_accuracy,
    #'f1_val': epoch_val_f1_macro,
    'epoch_val_losses':epoch_val_losses,
    'train_f1': train_f1,
    'train_acc': train_acc,
    'test_f1': test_f1,
    'test_acc': test_acc,
    'train_auc':train_auc,
    'test_auc':test_auc,
    'train_precision':train_precision,
    'train_recall':train_recall,
    'test_precision':test_precision,
    'test_recall':test_recall,
    'y_true_train':y_true_train,
    'y_true_val':y_true_val, #y_true_val
    'prediction_train': prediction_train,
    'prediction_test': prediction_test,
    'train_group':train_group,
    'test_group':val_group

})
    
detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


detailed_results_df.to_csv('multimodal_vgg.csv')
with open ('ecg_pcg'+dt_string+'test_results_finetuned_multimodal_vgg.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True}
Found 1793 validated image filenames belonging to 2 classes.
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Epoch 1/20
225/225 [==============================] - 40s 171ms/step - loss: 0.7835 - accuracy: 0.5878 - auc: 0.5734 - val_loss: 0.6752 - val_accuracy: 0.5566 - val_auc: 0.7122
Epoch 2/20
225/225 [==============================] - 38s 169ms/step - loss: 0.6014 - accuracy: 0.6492 - auc: 0.7286 - val_loss: 0.6281 - val_accuracy: 0.6632 - val_auc: 0.7410
Epoch 3/20
225/225 [==============================] - 38s 169ms/step - loss: 0.5138 - accuracy: 0.7262 - auc: 0.8188 - val_loss: 0.6154 - val_accuracy: 0.6723 - val_auc: 0.7608
Epoch 4/20
225/225 [==============================] - 38s 171ms/step - loss: 0.4333 - accuracy: 

In [20]:
##### # For each combination
param_dict ={'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': 'imagenet'}
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )
detailed_results = []
epoch_info = []
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
use_class_weights = param_dict['class_weights']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}

print(param_dict)
#print (f'-----------------FOLD {fold_idx}-----------------')

model = multimodal_imagenet(**param_dict_filtered)


ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
    dataframe=train_df_ecg,
    directory=f'{ecg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
    dataframe=train_df_pcg,
    directory=f'{pcg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

ecg_test_gen = test_datagen_ecg.flow_from_dataframe(
    dataframe=test_df_ecg,
    directory=f'{ecg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_test_gen = test_datagen_pcg.flow_from_dataframe(
    dataframe=test_df_pcg,
    directory=f'{pcg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')


paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)

train_group = train_df_ecg['group']
val_group = test_df_ecg['group']

if use_class_weights:
    y_train = ecg_train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
else:
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))


#paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
#paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))

ecg_train_gen.reset()

pcg_train_gen.reset()

ecg_test_gen.reset()

pcg_test_gen.reset()

paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)



test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))

#test_predictions = model.predict(test_gen)



print ( train_f1, train_acc, train_auc, train_precision,train_recall)

print (test_f1, test_acc, test_auc, test_precision, test_recall)



epoch_losses = history.history['loss']

epoch_accuracy = history.history['accuracy']
epoch_val_accuracy = history.history['val_accuracy']
epoch_val_losses = history.history['val_loss']

epoch_info.append({
    'params': param_dict,
    'fold_number': -1,
    'loss': epoch_losses,
    #'val_loss':epoch_val_losses,
    'acc_train': epoch_accuracy,
    #'f1_train': epoch_f1_macro,
    'acc_val': epoch_val_accuracy,
    #'f1_val': epoch_val_f1_macro,
    'epoch_val_losses':epoch_val_losses,
    'train_f1': train_f1,
    'train_acc': train_acc,
    'test_f1': test_f1,
    'test_acc': test_acc,
    'train_auc':train_auc,
    'test_auc':test_auc,
    'train_precision':train_precision,
    'train_recall':train_recall,
    'test_precision':test_precision,
    'test_recall':test_recall,
    'y_true_train':y_true_train,
    'y_true_val':y_true_val, #y_true_val
    'prediction_train': prediction_train,
    'prediction_test': prediction_test,
    'train_group':train_group,
    'test_group':val_group

})
    
detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


detailed_results_df.to_csv('multimodal_vgg.csv')
with open ('ecg_pcg'+dt_string+'test_results_imagenet_multimodal_vgg.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': 'imagenet'}
Found 1793 validated image filenames belonging to 2 classes.
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Epoch 1/20
225/225 [==============================] - 40s 170ms/step - loss: 1.4318 - accuracy: 0.5204 - auc: 0.5404 - val_loss: 0.7357 - val_accuracy: 0.5358 - val_auc: 0.6384
Epoch 2/20
225/225 [==============================] - 37s 166ms/step - loss: 0.7182 - accuracy: 0.5984 - auc: 0.6540 - val_loss: 0.7897 - val_accuracy: 0.4551 - val_auc: 0.7084
Epoch 3/20
225/225 [==============================] - 37s 164ms/step - loss: 0.6445 - accuracy: 0.6040 - auc: 0.7065 - val_loss: 0.6540 - val_accuracy: 0.6437 - val_auc: 0.7345
Epoch 4/20
225/225 [==============================] - 36s 161ms/step - lo

In [30]:
##### # For each combination
param_dict ={'batch_size': 8, 'epochs': 30, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 128, 'class_weights': False}
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )
detailed_results = []
epoch_info = []
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
use_class_weights = param_dict['class_weights']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}

print(param_dict)
#print (f'-----------------FOLD {fold_idx}-----------------')

model = multimodal_3(**param_dict_filtered)


ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
    dataframe=train_df_ecg,
    directory=f'{ecg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
    dataframe=train_df_pcg,
    directory=f'{pcg_folder}/train',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')

ecg_test_gen = test_datagen_ecg.flow_from_dataframe(
    dataframe=test_df_ecg,
    directory=f'{ecg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle = False, color_mode = 'rgb')

pcg_test_gen = test_datagen_pcg.flow_from_dataframe(
    dataframe=test_df_pcg,
    directory=f'{pcg_folder}/test',
    x_col='filename',
    y_col='label',
    class_mode = 'binary',
    classes = ["0","1"],
    target_size=(224, 224),
    batch_size=batch_size,
     shuffle = False, color_mode = 'rgb')


paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)

train_group = train_df_ecg['group']
val_group = test_df_ecg['group']

if use_class_weights:
    y_train = ecg_train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
else:
    history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))


#paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
#paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))

ecg_train_gen.reset()

pcg_train_gen.reset()

ecg_test_gen.reset()

pcg_test_gen.reset()

paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)


test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))

#test_predictions = model.predict(test_gen)



print ( train_f1, train_acc, train_auc, train_precision,train_recall)

print (test_f1, test_acc, test_auc, test_precision, test_recall)



epoch_losses = history.history['loss']

epoch_accuracy = history.history['accuracy']
epoch_val_accuracy = history.history['val_accuracy']
epoch_val_losses = history.history['val_loss']

epoch_info.append({
    'params': param_dict,
    'fold_number': -1,
    'loss': epoch_losses,
    #'val_loss':epoch_val_losses,
    'acc_train': epoch_accuracy,
    #'f1_train': epoch_f1_macro,
    'acc_val': epoch_val_accuracy,
    #'f1_val': epoch_val_f1_macro,
    'epoch_val_losses':epoch_val_losses,
    'train_f1': train_f1,
    'train_acc': train_acc,
    'test_f1': test_f1,
    'test_acc': test_acc,
    'train_auc':train_auc,
    'test_auc':test_auc,
    'train_precision':train_precision,
    'train_recall':train_recall,
    'test_precision':test_precision,
    'test_recall':test_recall,
    'y_true_train':y_true_train,
    'y_true_val':y_true_val, #y_true_val
    'prediction_train': prediction_train,
    'prediction_test': prediction_test,
    'train_group':train_group,
    'test_group':val_group

})
    
detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


detailed_results_df.to_csv('multimodal_vgg.csv')
with open ('ecg_pcg'+dt_string+'test_results_multimodal_vgg.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 30, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 128, 'class_weights': False}
Found 1793 validated image filenames belonging to 2 classes.
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Epoch 1/30
225/225 [==============================] - 83s 362ms/step - loss: 0.6579 - accuracy: 0.7072 - auc: 0.4767 - val_loss: 0.6202 - val_accuracy: 0.7178 - val_auc: 0.6522
Epoch 2/30
225/225 [==============================] - 81s 358ms/step - loss: 0.6198 - accuracy: 0.7083 - auc: 0.5010 - val_loss: 0.5834 - val_accuracy: 0.7178 - val_auc: 0.7236
Epoch 3/30
225/225 [==============================] - 81s 359ms/step - loss: 0.5971 - accuracy: 0.6972 - auc: 0.6086 - val_loss: 0.6260 - val_accuracy: 0.7178 - val_auc: 0.7266
Epoch 4/30
225/225 [==============================] - 81s 359ms/step - loss: 0.5722 - accuracy

In [25]:
param_grid = {
    'batch_size': [ 8],
    'epochs': [ 10],
    'optimizer': ['adam'],
    'learning_rate':  [  0.000001, 0.00001, 0.00005],
    'dropout':[  0.5],
    'neurons':[32],
    'class_weights':[True],
    'weights':[  'imagenet']
}



In [26]:
##### # For each combination

sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )

for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    use_class_weights = param_dict['class_weights']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_ecg['filename'], train_df_ecg['label'], groups=train_df_ecg['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = multimodal_imagenet(**param_dict_filtered)


        ecg_train_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[train_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_train_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[train_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        ecg_test_gen = train_datagen_ecg.flow_from_dataframe(
            dataframe=train_df_ecg.iloc[test_indices],
            directory=f'{ecg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
            shuffle = False, color_mode = 'rgb')

        pcg_test_gen = train_datagen_pcg.flow_from_dataframe(
            dataframe=train_df_pcg.iloc[test_indices],
            directory=f'{pcg_folder}/train',
            x_col='filename',
            y_col='label',
            class_mode = 'binary',
            classes = ["0","1"],
            target_size=(224, 224),
            batch_size=batch_size,
             shuffle = False, color_mode = 'rgb')

        
        paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        train_group = train_df_ecg.iloc[train_indices]['group']
        val_group = train_df_ecg.iloc[test_indices]['group']
        
        if use_class_weights:
            y_train = ecg_train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)
            
            
            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test,class_weight=class_weights, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        else:
            history = model.fit(paired_generator_train, batch_size = batch_size, epochs=n_epochs,validation_data=paired_generator_test, validation_steps=math.ceil(ecg_test_gen.n / ecg_test_gen.batch_size), steps_per_epoch = math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        
        #paired_generator_train = pair_generator(ecg_train_gen, pcg_train_gen )
        #paired_generator_test = pair_generator(ecg_test_gen, pcg_test_gen)
        
        
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(ecg_train_gen.classes, paired_generator_train, model, math.ceil(ecg_train_gen.n/ecg_train_gen.batch_size))
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(ecg_test_gen.classes, paired_generator_test, model, math.ceil(ecg_test_gen.n/  ecg_test_gen.batch_size))
        
        #test_predictions = model.predict(test_gen)
        
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['accuracy']
        epoch_val_accuracy = history.history['val_accuracy']
        epoch_val_losses = history.history['val_loss']

        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    detailed_results_df.to_csv('multimodal_vgg.csv')
    with open ('ecg_pcg'+dt_string+'imagenet_multimodal_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 8, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': 'imagenet'}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
180/180 [==============================] - 28s 151ms/step - loss: 9.2452 - accuracy: 0.6448 - auc: 0.4846 - val_loss: 2.6919 - val_accuracy: 0.6833 - val_auc: 0.4807
Epoch 2/10
180/180 [==============================] - 26s 147ms/step - loss: 5.5577 - accuracy: 0.5548 - auc: 0.4707 - val_loss: 1.9208 - val_accuracy: 0.5833 - val_auc: 0.5201
Epoch 3/10
180/180 [==============================] - 26s 147ms/step - loss: 4.3217 - accuracy: 0.4997 - auc: 0.4662 - val_loss: 1.8062 - val_accuracy: 0.5472 - val_auc: 0.5436
Epoch 4/10
180/180 [============

180/180 [==============================] - 26s 147ms/step - loss: 3.8578 - accuracy: 0.4896 - auc: 0.5164 - val_loss: 1.8931 - val_accuracy: 0.4594 - val_auc: 0.5158
Epoch 3/10
180/180 [==============================] - 26s 147ms/step - loss: 2.7292 - accuracy: 0.4979 - auc: 0.5343 - val_loss: 1.4791 - val_accuracy: 0.4874 - val_auc: 0.5288
Epoch 4/10
180/180 [==============================] - 26s 147ms/step - loss: 1.9125 - accuracy: 0.5515 - auc: 0.5741 - val_loss: 1.2598 - val_accuracy: 0.4958 - val_auc: 0.5231
Epoch 5/10
180/180 [==============================] - 26s 147ms/step - loss: 1.6404 - accuracy: 0.5070 - auc: 0.5626 - val_loss: 1.1999 - val_accuracy: 0.4734 - val_auc: 0.5183
Epoch 6/10
180/180 [==============================] - 26s 147ms/step - loss: 1.3525 - accuracy: 0.5494 - auc: 0.5844 - val_loss: 1.1384 - val_accuracy: 0.4622 - val_auc: 0.5242
Epoch 7/10
180/180 [==============================] - 26s 147ms/step - loss: 1.2641 - accuracy: 0.5362 - auc: 0.5940 - val_los

{'batch_size': 8, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': 'imagenet'}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
180/180 [==============================] - 28s 149ms/step - loss: 2.3340 - accuracy: 0.5583 - auc: 0.5313 - val_loss: 0.8617 - val_accuracy: 0.6417 - val_auc: 0.6537
Epoch 2/10
180/180 [==============================] - 26s 147ms/step - loss: 1.1001 - accuracy: 0.6190 - auc: 0.6307 - val_loss: 0.7170 - val_accuracy: 0.6361 - val_auc: 0.6792
Epoch 3/10
180/180 [==============================] - 26s 147ms/step - loss: 0.7957 - accuracy: 0.6636 - auc: 0.6985 - val_loss: 0.6450 - val_accuracy: 0.6722 - val_auc: 0.7068
Epoch 4/10
180/180 [============

180/180 [==============================] - 26s 147ms/step - loss: 1.0420 - accuracy: 0.5397 - auc: 0.5888 - val_loss: 0.9283 - val_accuracy: 0.5406 - val_auc: 0.6363
Epoch 3/10
180/180 [==============================] - 27s 147ms/step - loss: 0.9014 - accuracy: 0.5299 - auc: 0.6055 - val_loss: 0.9317 - val_accuracy: 0.4902 - val_auc: 0.6425
Epoch 4/10
180/180 [==============================] - 26s 147ms/step - loss: 0.7414 - accuracy: 0.5543 - auc: 0.6575 - val_loss: 0.8914 - val_accuracy: 0.5014 - val_auc: 0.6589
Epoch 5/10
180/180 [==============================] - 26s 147ms/step - loss: 0.6720 - accuracy: 0.6177 - auc: 0.6978 - val_loss: 0.7706 - val_accuracy: 0.5574 - val_auc: 0.6670
Epoch 6/10
180/180 [==============================] - 27s 148ms/step - loss: 0.6143 - accuracy: 0.6588 - auc: 0.7270 - val_loss: 0.7120 - val_accuracy: 0.5854 - val_auc: 0.6842
Epoch 7/10
180/180 [==============================] - 27s 149ms/step - loss: 0.5762 - accuracy: 0.6776 - auc: 0.7645 - val_los

Found 1433 validated image filenames belonging to 2 classes.
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
180/180 [==============================] - 28s 149ms/step - loss: 1.6345 - accuracy: 0.5283 - auc: 0.5897 - val_loss: 0.7108 - val_accuracy: 0.4778 - val_auc: 0.7061
Epoch 2/10
180/180 [==============================] - 26s 147ms/step - loss: 0.6522 - accuracy: 0.6001 - auc: 0.6666 - val_loss: 0.6754 - val_accuracy: 0.5639 - val_auc: 0.7678
Epoch 3/10
180/180 [==============================] - 26s 147ms/step - loss: 0.5826 - accuracy: 0.6518 - auc: 0.7558 - val_loss: 0.5965 - val_accuracy: 0.5722 - val_auc: 0.7641
Epoch 4/10
180/180 [==============================] - 26s 147ms/step - loss: 0.5573 - accuracy: 0.6916 - auc: 0.7988 - val_loss: 0.6418 - val_accuracy: 0.5750 - val_auc: 0.7044
Epoch 5/10
180/180 [==============================

180/180 [==============================] - 26s 147ms/step - loss: 0.7100 - accuracy: 0.7173 - auc: 0.7891 - val_loss: 0.5312 - val_accuracy: 0.7535 - val_auc: 0.8605
Epoch 4/10
180/180 [==============================] - 27s 147ms/step - loss: 0.5529 - accuracy: 0.7451 - auc: 0.8439 - val_loss: 0.7215 - val_accuracy: 0.6919 - val_auc: 0.8714
Epoch 5/10
180/180 [==============================] - 26s 147ms/step - loss: 0.4138 - accuracy: 0.8106 - auc: 0.9044 - val_loss: 0.5759 - val_accuracy: 0.7619 - val_auc: 0.8892
Epoch 6/10
180/180 [==============================] - 26s 147ms/step - loss: 0.3732 - accuracy: 0.8210 - auc: 0.9186 - val_loss: 0.6735 - val_accuracy: 0.7199 - val_auc: 0.8934
Epoch 7/10
180/180 [==============================] - 26s 147ms/step - loss: 0.3002 - accuracy: 0.8552 - auc: 0.9461 - val_loss: 0.5784 - val_accuracy: 0.7759 - val_auc: 0.9024
Epoch 8/10
180/180 [==============================] - 26s 147ms/step - loss: 0.2894 - accuracy: 0.8593 - auc: 0.9511 - val_los

In [28]:
# Define your model creation function
def create_model_single_mode(optimizer='adam', learning_rate=0.001, dropout = 0.5, neurons = 128, weights=None):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
    base_model.trainable = True

   
    
    
        
        
    x = base_model.output
    
    
    x = Flatten()(x)  
    x = Dense(neurons, activation='relu')(x)  # Add a fully connected layer
    x = Dropout(dropout) (x)
    predictions = Dense(1, activation='sigmoid')(x)  # Replace softmax with sigmoid for binary classification

    model = Model(inputs=base_model.input, outputs=predictions)
    
        
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['Accuracy', 'AUC'])
    return model

In [29]:
def calculate_scores_single(generator, model):
    y_true = generator.classes
    predictions = model.predict(generator)
    threshold = 0.5
    predicted_classes = (predictions > threshold).astype(int)
    
    f1_macro_value = f1_score(y_true,predicted_classes, average='binary' )
    acc = accuracy_score(y_true, predicted_classes)
    auc = roc_auc_score(y_true, predictions) 
    precision = precision_score(y_true, predicted_classes)
    recall = recall_score(y_true, predicted_classes)
    return f1_macro_value, acc, auc, precision, recall, predictions, y_true

## ECG Single mode

In [22]:
param_grid = {
    'batch_size': [ 32],
    'epochs': [10],
    'optimizer': ['adam'],
    'learning_rate':  [ 0.000001, 0.00001, 0.00005],
    'dropout':[  0.5],
    'neurons':[32],
    'class_weights':[True],
    'weights':[  None]
}


In [23]:
##### # For each combination
#import time
#time.sleep(8000)
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )
for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    use_class_weights = param_dict['class
                                   _weights']
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_ecg['filename'], train_df_ecg['label'], groups=train_df_ecg['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = create_model_single_mode(**param_dict_filtered)


        train_gen = train_datagen_ecg.flow_from_dataframe(dataframe=train_df_ecg.iloc[train_indices],
                                             directory=f"./{ecg_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                             color_mode = 'rgb' )



        test_gen = train_datagen_ecg.flow_from_dataframe(dataframe=train_df_ecg.iloc[test_indices],
                                             directory=f"./{ecg_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             shuffle = False,
                                             classes = ["0","1"], batch_size=batch_size,
                                             color_mode = 'rgb')
        
        train_group = train_df_ecg.iloc[train_indices]['group']
        val_group = train_df_ecg.iloc[test_indices]['group']
        #score_callback = ScoreCallback(test_gen, train_gen)
        
        #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
        if use_class_weights:
            y_train = train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)


            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen,class_weight=class_weights)
        else:
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen)
                
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores_single(train_gen, model)
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores_single(test_gen, model)
        
        #test_predictions = model.predict(test_gen)
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        #print ('F1-macro:', test_score)
         # Access loss, accuracy, and f1_macro values from history
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['Accuracy']
        epoch_val_accuracy = history.history['val_Accuracy']
        #epoch_accuracies = score_callback.accuracies_train
        #epoch_f1_macro = score_callback.train_f1_scores
        epoch_val_losses = history.history['val_loss']
        #epoch_val_accuracies = score_callback.accuracies_val
        #epoch_val_f1_macro = score_callback.val_f1_scores
        # After training
        #print("F1-scores per epoch:", score_callback.val_f1_scores)
        #print("Accuracies per epoch:", accuracy_callback.accuracies)
        
        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    #detailed_results_df.to_csv(main_folder+'_vgg.csv')
    with open ('ecg_scratch'+dt_string+'ecg_scratch_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 37s 639ms/step - loss: 5.7535 - Accuracy: 0.4655 - auc: 0.4719 - val_loss: 1.4270 - val_Accuracy: 0.6139 - val_auc: 0.6788
Epoch 2/10
45/45 [==============================] - 23s 508ms/step - loss: 2.9074 - Accuracy: 0.5611 - auc: 0.5603 - val_loss: 1.1894 - val_Accuracy: 0.6472 - val_auc: 0.7157
Epoch 3/10
45/45 [==============================] - 22s 499ms/step - loss: 1.9420 - Accuracy: 0.5534 - auc: 0.5987 - val_loss: 0.8597 - val_Accuracy: 0.6694 - val_auc: 0.7200
Epoch 4/10
45/45 [==============================] - 23s 501ms/step - loss: 1.4751 - Accuracy: 0.5841 - auc: 0.6322 - val_loss: 0.7929 - val_Accuracy: 0.6389 - val_a

45/45 [==============================] - 23s 507ms/step - loss: 1.0360 - Accuracy: 0.6393 - auc: 0.6712 - val_loss: 0.7838 - val_Accuracy: 0.6555 - val_auc: 0.7056
Epoch 6/10
45/45 [==============================] - 23s 506ms/step - loss: 0.9089 - Accuracy: 0.6379 - auc: 0.6972 - val_loss: 0.7723 - val_Accuracy: 0.6387 - val_auc: 0.7266
Epoch 7/10
45/45 [==============================] - 23s 507ms/step - loss: 0.8684 - Accuracy: 0.6504 - auc: 0.6889 - val_loss: 0.7275 - val_Accuracy: 0.6583 - val_auc: 0.7355
Epoch 8/10
45/45 [==============================] - 23s 517ms/step - loss: 0.7751 - Accuracy: 0.6706 - auc: 0.7190 - val_loss: 0.6771 - val_Accuracy: 0.6695 - val_auc: 0.7528
Epoch 9/10
45/45 [==============================] - 23s 508ms/step - loss: 0.6781 - Accuracy: 0.6943 - auc: 0.7561 - val_loss: 0.6490 - val_Accuracy: 0.6779 - val_auc: 0.7562
Epoch 10/10
12/12 [==============================] - 2s 166ms/step
0.8355650776647028 0.7862116991643454 0.8826420134983127 0.9165687426

Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 24s 525ms/step - loss: 1.8165 - Accuracy: 0.4906 - auc: 0.5318 - val_loss: 0.6808 - val_Accuracy: 0.5611 - val_auc: 0.5446
Epoch 2/10
45/45 [==============================] - 24s 524ms/step - loss: 0.6753 - Accuracy: 0.5604 - auc: 0.6317 - val_loss: 0.5966 - val_Accuracy: 0.6389 - val_auc: 0.7159
Epoch 3/10
45/45 [==============================] - 23s 513ms/step - loss: 0.5989 - Accuracy: 0.6427 - auc: 0.7328 - val_loss: 0.5618 - val_Accuracy: 0.6917 - val_auc: 0.7316
Epoch 4/10
45/45 [==============================] - 23s 520ms/step - loss: 0.5653 - Accuracy: 0.6504 - auc: 0.7658 - val_loss: 0.6928 - val_Accuracy: 0.5528 - val_auc: 0.7545
Epoch 5/10
45/45 [==============================] - 23s 512ms/step - loss: 0.5261 - Accuracy: 0.7104 - auc: 0.8106 - val_loss: 0.5462 - val_Accuracy: 0.6833 - val_auc: 0.7895
Epoc

Epoch 7/10
45/45 [==============================] - 26s 565ms/step - loss: 0.4445 - Accuracy: 0.7597 - auc: 0.8698 - val_loss: 0.4359 - val_Accuracy: 0.8151 - val_auc: 0.8572
Epoch 8/10
45/45 [==============================] - 23s 510ms/step - loss: 0.4100 - Accuracy: 0.8001 - auc: 0.8887 - val_loss: 0.3918 - val_Accuracy: 0.8487 - val_auc: 0.9063
Epoch 9/10
45/45 [==============================] - 23s 511ms/step - loss: 0.3826 - Accuracy: 0.8099 - auc: 0.9051 - val_loss: 0.3866 - val_Accuracy: 0.8207 - val_auc: 0.9391
Epoch 10/10
12/12 [==============================] - 2s 186ms/step
0.9423376623376624 0.9227019498607242 0.9926696662917135 0.9977997799779978 0.8927165354330708
0.8846960167714885 0.84593837535014 0.9247756806930694 0.9547511312217195 0.82421875
{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 4-----------------
Found 1442 validated image filenames b

45/45 [==============================] - 47s 1s/step - loss: 1.6737 - Accuracy: 0.5080 - auc: 0.5013 - val_loss: 0.6861 - val_Accuracy: 0.5972 - val_auc: 0.5543
Epoch 2/10
45/45 [==============================] - 37s 825ms/step - loss: 0.7079 - Accuracy: 0.4648 - auc: 0.4918 - val_loss: 0.6397 - val_Accuracy: 0.7083 - val_auc: 0.6402
Epoch 3/10
45/45 [==============================] - 38s 837ms/step - loss: 0.6839 - Accuracy: 0.5052 - auc: 0.5616 - val_loss: 0.6310 - val_Accuracy: 0.7056 - val_auc: 0.6577
Epoch 4/10
45/45 [==============================] - 40s 881ms/step - loss: 0.6857 - Accuracy: 0.5164 - auc: 0.5722 - val_loss: 0.6526 - val_Accuracy: 0.6806 - val_auc: 0.6587
Epoch 5/10
45/45 [==============================] - 36s 807ms/step - loss: 0.6865 - Accuracy: 0.5359 - auc: 0.5544 - val_loss: 0.6268 - val_Accuracy: 0.5611 - val_auc: 0.6626
Epoch 6/10
45/45 [==============================] - 37s 825ms/step - loss: 0.6616 - Accuracy: 0.5813 - auc: 0.6227 - val_loss: 0.6788 - val

45/45 [==============================] - 23s 504ms/step - loss: 0.6167 - Accuracy: 0.6553 - auc: 0.7228 - val_loss: 0.6740 - val_Accuracy: 0.5630 - val_auc: 0.8332
Epoch 8/10
45/45 [==============================] - 23s 504ms/step - loss: 0.7393 - Accuracy: 0.5961 - auc: 0.6724 - val_loss: 0.7402 - val_Accuracy: 0.4230 - val_auc: 0.7972
Epoch 9/10
45/45 [==============================] - 23s 503ms/step - loss: 0.6469 - Accuracy: 0.6107 - auc: 0.6735 - val_loss: 0.5647 - val_Accuracy: 0.7731 - val_auc: 0.7888
Epoch 10/10
12/12 [==============================] - 2s 165ms/step
0.7715617715617715 0.7270194986072424 0.8611454818147731 0.9457142857142857 0.6515748031496063
0.6600496277915633 0.6162464985994398 0.8212407178217822 0.9047619047619048 0.51953125
{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 4-----------------
Found 1442 validated image filenames belonging 

## PCG Single mode

In [24]:
param_grid = {
    'batch_size': [ 32],
    'epochs': [10],
    'optimizer': ['adam'],
    'learning_rate':  [ 0.000001, 0.00001, 0.00005],
    'dropout':[  0.5],
    'neurons':[32],
    'class_weights':[True],
    'weights':[  None]
}

In [25]:
##### # For each combination
#import time
#time.sleep(8000)
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )
for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    use_class_weights = param_dict['class_weights']
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_pcg['filename'],train_df_pcg['label'], groups=train_df_pcg['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = create_model_single_mode(**param_dict_filtered)


        train_gen = train_datagen_pcg.flow_from_dataframe(dataframe=train_df_pcg.iloc[train_indices],
                                             directory=f"./{pcg_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                             color_mode = 'rgb' )



        test_gen = train_datagen_pcg.flow_from_dataframe(dataframe=train_df_pcg.iloc[test_indices],
                                             directory=f"./{pcg_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             shuffle = False,
                                             classes = ["0","1"], batch_size=batch_size,
                                             color_mode = 'rgb')
        
        train_group = train_df_pcg.iloc[train_indices]['group']
        val_group = train_df_pcg.iloc[test_indices]['group']
        #score_callback = ScoreCallback(test_gen, train_gen)
        
        #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
        if use_class_weights:
            y_train = train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)


            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen,class_weight=class_weights)
        else:
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen)
                
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores_single(train_gen, model)
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores_single(test_gen, model)
        
        #test_predictions = model.predict(test_gen)
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        #print ('F1-macro:', test_score)
         # Access loss, accuracy, and f1_macro values from history
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['Accuracy']
        epoch_val_accuracy = history.history['val_Accuracy']
        #epoch_accuracies = score_callback.accuracies_train
        #epoch_f1_macro = score_callback.train_f1_scores
        epoch_val_losses = history.history['val_loss']
        #epoch_val_accuracies = score_callback.accuracies_val
        #epoch_val_f1_macro = score_callback.val_f1_scores
        # After training
        #print("F1-scores per epoch:", score_callback.val_f1_scores)
        #print("Accuracies per epoch:", accuracy_callback.accuracies)
        
        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    detailed_results_df.to_csv(pcg_folder+'_vgg.csv')
    with open (pcg_folder+dt_string+'pcg_scratch_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 0-----------------
Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 25s 524ms/step - loss: 4.9493 - Accuracy: 0.5255 - auc: 0.5244 - val_loss: 2.1106 - val_Accuracy: 0.4500 - val_auc: 0.4774
Epoch 2/10
45/45 [==============================] - 23s 510ms/step - loss: 2.6118 - Accuracy: 0.5080 - auc: 0.5226 - val_loss: 1.3772 - val_Accuracy: 0.4278 - val_auc: 0.5001
Epoch 3/10
45/45 [==============================] - 23s 511ms/step - loss: 1.8152 - Accuracy: 0.5262 - auc: 0.5307 - val_loss: 0.9841 - val_Accuracy: 0.5611 - val_auc: 0.5254
Epoch 4/10
45/45 [==============================] - 23s 516ms/step - loss: 1.3575 - Accuracy: 0.5464 - auc: 0.5669 - val_loss: 0.9059 - val_Accuracy: 0.5722 - val_a

45/45 [==============================] - 23s 504ms/step - loss: 0.9980 - Accuracy: 0.5209 - auc: 0.5091 - val_loss: 0.7847 - val_Accuracy: 0.5378 - val_auc: 0.4605
Epoch 6/10
45/45 [==============================] - 23s 503ms/step - loss: 0.8494 - Accuracy: 0.5397 - auc: 0.5352 - val_loss: 0.7344 - val_Accuracy: 0.5462 - val_auc: 0.4734
Epoch 7/10
45/45 [==============================] - 23s 503ms/step - loss: 0.7934 - Accuracy: 0.5474 - auc: 0.5435 - val_loss: 0.7057 - val_Accuracy: 0.5602 - val_auc: 0.4968
Epoch 8/10
45/45 [==============================] - 23s 504ms/step - loss: 0.7633 - Accuracy: 0.5383 - auc: 0.5501 - val_loss: 0.6849 - val_Accuracy: 0.5798 - val_auc: 0.5126
Epoch 9/10
45/45 [==============================] - 23s 504ms/step - loss: 0.7200 - Accuracy: 0.5494 - auc: 0.5641 - val_loss: 0.6862 - val_Accuracy: 0.5742 - val_auc: 0.5202
Epoch 10/10
12/12 [==============================] - 2s 165ms/step
0.7493893502686858 0.6427576601671309 0.6444389763779528 0.7439379243

Found 1433 validated image filenames belonging to 2 classes.
Found 360 validated image filenames belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 24s 511ms/step - loss: 1.9838 - Accuracy: 0.5192 - auc: 0.4873 - val_loss: 0.7177 - val_Accuracy: 0.4972 - val_auc: 0.5190
Epoch 2/10
45/45 [==============================] - 23s 503ms/step - loss: 0.7106 - Accuracy: 0.4599 - auc: 0.5569 - val_loss: 0.7590 - val_Accuracy: 0.3278 - val_auc: 0.4913
Epoch 3/10
45/45 [==============================] - 23s 503ms/step - loss: 0.6754 - Accuracy: 0.5213 - auc: 0.5814 - val_loss: 0.7012 - val_Accuracy: 0.4333 - val_auc: 0.5381
Epoch 4/10
45/45 [==============================] - 23s 503ms/step - loss: 0.6732 - Accuracy: 0.3992 - auc: 0.5953 - val_loss: 0.7075 - val_Accuracy: 0.4111 - val_auc: 0.5172
Epoch 5/10
45/45 [==============================] - 23s 504ms/step - loss: 0.6694 - Accuracy: 0.5178 - auc: 0.6114 - val_loss: 0.7531 - val_Accuracy: 0.4861 - val_auc: 0.5511
Epoc

Epoch 7/10
45/45 [==============================] - 24s 529ms/step - loss: 0.6783 - Accuracy: 0.4582 - auc: 0.5991 - val_loss: 0.6708 - val_Accuracy: 0.5742 - val_auc: 0.6159
Epoch 8/10
45/45 [==============================] - 23s 502ms/step - loss: 0.6685 - Accuracy: 0.4909 - auc: 0.5973 - val_loss: 0.7092 - val_Accuracy: 0.3922 - val_auc: 0.6272
Epoch 9/10
45/45 [==============================] - 28s 617ms/step - loss: 0.6522 - Accuracy: 0.4547 - auc: 0.6118 - val_loss: 0.6586 - val_Accuracy: 0.6050 - val_auc: 0.6548
Epoch 10/10
12/12 [==============================] - 2s 165ms/step
0.7762605042016807 0.7033426183844012 0.7691272965879266 0.8322072072072072 0.7273622047244095
0.7019027484143763 0.6050420168067226 0.5866336633663366 0.7649769585253456 0.6484375
{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 4-----------------
Found 1442 validated image filenames 

45/45 [==============================] - 30s 640ms/step - loss: 1.2082 - Accuracy: 0.5541 - auc: 0.5513 - val_loss: 0.6313 - val_Accuracy: 0.7250 - val_auc: 0.4580
Epoch 2/10
45/45 [==============================] - 24s 530ms/step - loss: 0.7207 - Accuracy: 0.5038 - auc: 0.4818 - val_loss: 0.6531 - val_Accuracy: 0.7250 - val_auc: 0.4807
Epoch 3/10
45/45 [==============================] - 23s 514ms/step - loss: 0.7021 - Accuracy: 0.3810 - auc: 0.4579 - val_loss: 0.7016 - val_Accuracy: 0.2750 - val_auc: 0.5049
Epoch 4/10
45/45 [==============================] - 23s 508ms/step - loss: 0.6942 - Accuracy: 0.3238 - auc: 0.4817 - val_loss: 0.7023 - val_Accuracy: 0.2750 - val_auc: 0.4806
Epoch 5/10
45/45 [==============================] - 25s 549ms/step - loss: 0.6942 - Accuracy: 0.2959 - auc: 0.4778 - val_loss: 0.7002 - val_Accuracy: 0.2750 - val_auc: 0.4788
Epoch 6/10
45/45 [==============================] - 24s 526ms/step - loss: 0.6934 - Accuracy: 0.3029 - auc: 0.5013 - val_loss: 0.6993 - 

45/45 [==============================] - 23s 511ms/step - loss: 0.6994 - Accuracy: 0.6205 - auc: 0.4811 - val_loss: 0.6856 - val_Accuracy: 0.6695 - val_auc: 0.6139
Epoch 8/10
45/45 [==============================] - 23s 510ms/step - loss: 0.6911 - Accuracy: 0.5383 - auc: 0.5388 - val_loss: 0.6745 - val_Accuracy: 0.7283 - val_auc: 0.5838
Epoch 9/10
45/45 [==============================] - 23s 510ms/step - loss: 0.6960 - Accuracy: 0.5049 - auc: 0.4741 - val_loss: 0.6718 - val_Accuracy: 0.7367 - val_auc: 0.5844
Epoch 10/10
12/12 [==============================] - 2s 174ms/step
0.325508607198748 0.3997214484679666 0.6219839707536559 0.7938931297709924 0.2047244094488189
0.3478260869565217 0.4117647058823529 0.652034344059406 0.8484848484848485 0.21875
{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 5e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 4-----------------
Found 1442 validated image filenames belonging to 2 

In [26]:
param_dict ={'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': False, 'weights': None}

detailed_results = []
#param_dict = dict(zip(param_grid.keys(), params))
use_class_weights = param_dict['class_weights']
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}

    
epoch_info = []


print(param_dict)
print (f'-----------------FOLD {fold_idx}-----------------')

model = create_model_single_mode(**param_dict_filtered)


train_gen = train_datagen_pcg.flow_from_dataframe(dataframe=train_df_pcg,
                                     directory=f"./{pcg_folder}/train",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                     color_mode = 'rgb' )



test_gen = test_datagen_pcg.flow_from_dataframe(dataframe=test_df_pcg,
                                     directory=f"./{pcg_folder}/test",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     shuffle = False,
                                     classes = ["0","1"], batch_size=batch_size,
                                     color_mode = 'rgb')

train_group = train_df_pcg['group']
val_group = test_df_pcg['group']
#score_callback = ScoreCallback(test_gen, train_gen)

#history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
if use_class_weights:
    y_train = train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen,class_weight=class_weights)
else:
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen)

train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores_single(train_gen, model)

test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores_single(test_gen, model)

#test_predictions = model.predict(test_gen)


print ( train_f1, train_acc, train_auc, train_precision,train_recall)

print (test_f1, test_acc, test_auc, test_precision, test_recall)



#print ('F1-macro:', test_score)
 # Access loss, accuracy, and f1_macro values from history
epoch_losses = history.history['loss']

epoch_accuracy = history.history['Accuracy']
epoch_val_accuracy = history.history['val_Accuracy']
#epoch_accuracies = score_callback.accuracies_train
#epoch_f1_macro = score_callback.train_f1_scores
epoch_val_losses = history.history['val_loss']
#epoch_val_accuracies = score_callback.accuracies_val
#epoch_val_f1_macro = score_callback.val_f1_scores
# After training
#print("F1-scores per epoch:", score_callback.val_f1_scores)
#print("Accuracies per epoch:", accuracy_callback.accuracies)

epoch_info.append({
    'params': param_dict,
    'fold_number': -1,
    'loss': epoch_losses,
    #'val_loss':epoch_val_losses,
    'acc_train': epoch_accuracy,
    #'f1_train': epoch_f1_macro,
    'acc_val': epoch_val_accuracy,
    #'f1_val': epoch_val_f1_macro,
    'epoch_val_losses':epoch_val_losses,
    'train_f1': train_f1,
    'train_acc': train_acc,
    'test_f1': test_f1,
    'test_acc': test_acc,
    'train_auc':train_auc,
    'test_auc':test_auc,
    'train_precision':train_precision,
    'train_recall':train_recall,
    'test_precision':test_precision,
    'test_recall':test_recall,
    'y_true_train':y_true_train,
    'y_true_val':y_true_val, #y_true_val
    'prediction_train': prediction_train,
    'prediction_test': prediction_test,
    'train_group':train_group,
    'test_group':val_group

})

detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


#detailed_results_df.to_csv(main_folder+'_vgg.csv')
with open ('pcg_scratch'+dt_string+'test_results.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)



{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': False, 'weights': None}
-----------------FOLD 4-----------------
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Epoch 1/10
57/57 [==============================] - 32s 549ms/step - loss: 1.1386 - Accuracy: 0.6113 - auc: 0.5373 - val_loss: 0.5810 - val_Accuracy: 0.7269 - val_auc: 0.6083
Epoch 2/10
57/57 [==============================] - 31s 536ms/step - loss: 0.6530 - Accuracy: 0.6832 - auc: 0.5361 - val_loss: 0.5959 - val_Accuracy: 0.7269 - val_auc: 0.6424
Epoch 3/10
57/57 [==============================] - 30s 535ms/step - loss: 0.6302 - Accuracy: 0.7111 - auc: 0.5557 - val_loss: 0.5738 - val_Accuracy: 0.7334 - val_auc: 0.6783
Epoch 4/10
57/57 [==============================] - 31s 540ms/step - loss: 0.6231 - Accuracy: 0.7144 - auc: 0.5900 - val_loss: 0.5605 - val_Accuracy: 0.7269 - val_

In [30]:
param_dict = {'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
detailed_results = []
#param_dict = dict(zip(param_grid.keys(), params))
use_class_weights = param_dict['class_weights']
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}

    
epoch_info = []


print(param_dict)
print (f'-----------------FOLD {fold_idx}-----------------')

model = create_model_single_mode(**param_dict_filtered)


train_gen = train_datagen_ecg.flow_from_dataframe(dataframe=train_df_ecg,
                                     directory=f"./{ecg_folder}/train",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                     color_mode = 'rgb' )



test_gen = test_datagen_ecg.flow_from_dataframe(dataframe=test_df_ecg,
                                     directory=f"./{ecg_folder}/test",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     shuffle = False,
                                     classes = ["0","1"], batch_size=batch_size,
                                     color_mode = 'rgb')

train_group = train_df_ecg['group']
val_group = test_df_ecg['group']
#score_callback = ScoreCallback(test_gen, train_gen)

#history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
if use_class_weights:
    y_train = train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen,class_weight=class_weights)
else:
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen)

train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores_single(train_gen, model)

test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores_single(test_gen, model)

#test_predictions = model.predict(test_gen)


print ( train_f1, train_acc, train_auc, train_precision,train_recall)

print (test_f1, test_acc, test_auc, test_precision, test_recall)



#print ('F1-macro:', test_score)
 # Access loss, accuracy, and f1_macro values from history
epoch_losses = history.history['loss']

epoch_accuracy = history.history['Accuracy']
epoch_val_accuracy = history.history['val_Accuracy']
#epoch_accuracies = score_callback.accuracies_train
#epoch_f1_macro = score_callback.train_f1_scores
epoch_val_losses = history.history['val_loss']
#epoch_val_accuracies = score_callback.accuracies_val
#epoch_val_f1_macro = score_callback.val_f1_scores
# After training
#print("F1-scores per epoch:", score_callback.val_f1_scores)
#print("Accuracies per epoch:", accuracy_callback.accuracies)

epoch_info.append({
    'params': param_dict,
    'fold_number': -1,
    'loss': epoch_losses,
    #'val_loss':epoch_val_losses,
    'acc_train': epoch_accuracy,
    #'f1_train': epoch_f1_macro,
    'acc_val': epoch_val_accuracy,
    #'f1_val': epoch_val_f1_macro,
    'epoch_val_losses':epoch_val_losses,
    'train_f1': train_f1,
    'train_acc': train_acc,
    'test_f1': test_f1,
    'test_acc': test_acc,
    'train_auc':train_auc,
    'test_auc':test_auc,
    'train_precision':train_precision,
    'train_recall':train_recall,
    'test_precision':test_precision,
    'test_recall':test_recall,
    'y_true_train':y_true_train,
    'y_true_val':y_true_val, #y_true_val
    'prediction_train': prediction_train,
    'prediction_test': prediction_test,
    'train_group':train_group,
    'test_group':val_group

})

detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


#detailed_results_df.to_csv(main_folder+'_vgg.csv')
with open ('ecg_scratch'+dt_string+'test_results.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)


{'batch_size': 32, 'epochs': 10, 'optimizer': 'adam', 'learning_rate': 1e-05, 'dropout': 0.5, 'neurons': 32, 'class_weights': True, 'weights': None}
-----------------FOLD 4-----------------
Found 1793 validated image filenames belonging to 2 classes.
Found 769 validated image filenames belonging to 2 classes.
Epoch 1/10
57/57 [==============================] - 42s 596ms/step - loss: 1.7248 - Accuracy: 0.5209 - auc: 0.5503 - val_loss: 0.5610 - val_Accuracy: 0.7334 - val_auc: 0.7721
Epoch 2/10
57/57 [==============================] - 32s 570ms/step - loss: 0.6519 - Accuracy: 0.5839 - auc: 0.6839 - val_loss: 0.4975 - val_Accuracy: 0.7659 - val_auc: 0.8201
Epoch 3/10
57/57 [==============================] - 32s 570ms/step - loss: 0.5465 - Accuracy: 0.6871 - auc: 0.7946 - val_loss: 0.4726 - val_Accuracy: 0.7711 - val_auc: 0.8037
Epoch 4/10
57/57 [==============================] - 32s 569ms/step - loss: 0.4851 - Accuracy: 0.7306 - auc: 0.8421 - val_loss: 0.4526 - val_Accuracy: 0.7841 - val_a

In [26]:
len(test_df_ecg)

769